# Purpose
This script is for generating the posting units and tags from the raw data set and feed into the inverted index

In [1]:
import os
os.chdir('..')  # change current path to the root of project

from inv_index import Index
from probes import Index_probe
from data_structure import Post_unit
import pandas as pd
import global_settings as gs
from collections import Counter
import re

os.path.abspath(os.curdir)

'C:\\desktop\\workspace\\moive_recommendation'

# Load the raw data  
1. genome-scores.csv  
2. genome-tags.csv  
3. tags.csv  

In [2]:
genScorePath = os.path.join(gs.datasetPath, 'genome-scores.csv')
# genTagPath = os.path.join(gs.datasetPath, 'genome-tags.csv')
tagPath = os.path.join(gs.datasetPath, 'tags.csv')
moviePath = os.path.join(gs.datasetPath, 'movies.csv')

In [3]:
genScoreDF = pd.read_csv(genScorePath, delimiter = ',')
# genTagDF = pd.read_csv(genTagPath, delimiter = ',')
tagDF = pd.read_csv(tagPath, delimiter = ',')
movieDF = pd.read_csv(moviePath, delimiter = ',')

In [4]:
tagDF.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [5]:
len(tagDF)

465564

In [6]:
# not all the movieId are presenting - not all movie have tags
tagDF.movieId.max() - tagDF.movieId.unique().shape

array([111713])

In [7]:
movieDF.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
len(movieDF)

27278

In [9]:
# not all movie have tag genome scores
genScoreDF.movieId.unique().shape

(10381,)

# Transfer the raw data into posting unit format  
*userId,movieId,tag,timestamp -> tagText, currentId, nextId, previousId, uPropJson, moiveId, status*  
**Note**: same tags made by different user are regard as different occurances, so that the tf is set


In [11]:
def tagClean(tagText):
    tagText = re.sub('[^a-zA-Z _]', '', tagText)  # eliminate symbols which are not letters or space or _
    tagText = tagText.replace(' ', '_')
    if tagText.upper() in ['CON', 'PRN', 'AUX', 'NUL', 'COM1', 'COM2', 'COM3', 'COM4', 'COM5', 'COM6', 'COM7', 'COM8', 'COM9', 'LPT1', 'LPT2', 'LPT3', 'LPT4', 'LPT5', 'LPT6', 'LPT7', 'LPT8', 'LPT9']:
        tagText = '_%s_'%tagText
    return tagText.lower()

In [12]:
# collecting the occurance of tags in terms of one movie
movieTagCounter = Counter()  
longTagCount = 0
for record in tagDF.values:
    movieId = record[1]
    tagText = str(record[2])
    if tagText.count(' ') > 5:  
        longTagCount += 1
    else:
        # if the tag is too long, eliminate it
        tagText = tagClean(tagText)
        if tagText != '':
            movieTagCounter[(movieId, tagText)] += 1

In [13]:
# 19991 out of 465,564 tags are long tags, just leave them there.
longTagCount

1651

In [14]:
flatPUnitList = []  # for collecting the flatten posting units

In [15]:
# tagText, currentId, nextId, previousId, uPropJson, moiveId, status
for movieTagPair in movieTagCounter:
    flatPUnitList.append("%s -1 -1 -1 {\"tf\":%d} %d 1"%(movieTagPair[1], movieTagCounter[movieTagPair], movieTagPair[0]))

# Feed the flatten posting units in to inverted index

In [16]:
idx = Index()

In [17]:
%%time
for flatPUnit in flatPUnitList:
    pUnit = Post_unit.deflatten(flatPUnit)    
    idx.add_post_unit(pUnit)
    
    # add the corresponding movie info into the index
    if not pUnit.moiveId in idx.docInfo:
        movieInfo = movieDF.loc[movieDF.movieId == pUnit.moiveId].values.ravel()
        title = movieInfo[1]
        genre = movieInfo[2]
        movieInfoDict = {}
        movieInfoDict['title'] = title
        movieInfoDict['genre'] = genre

        idx.add_doc_info(pUnit.moiveId, movieInfoDict)

Wall time: 13.6 s


In [18]:
idx.persist_index()

2018-12-01 15:22:26,223 [INFO] - Index.py:95 persist last post unit ID
2018-12-01 15:22:26,228 [INFO] - Index.py:68 persist lexicon
2018-12-01 15:22:26,556 [INFO] - Index.py:77 persist posting
2018-12-01 15:23:30,509 [INFO] - Index.py:102 persist doc info
